In [1]:
import sparknlp 

spark = sparknlp.start()

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Spark NLP version
Apache Spark version


'3.1.1'

In [2]:
from sparknlp.pretrained import PretrainedPipeline
pipeline = PretrainedPipeline('analyze_sentiment', 'en')
result = pipeline.annotate('This is a very good movie. I recommend others to awoid this, movie is not good.')
print(result['sentiment'])

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]
['positive', 'negative']


In [4]:
print(result['checked'])

['This', 'is', 'a', 'very', 'good', 'movie', '.', 'I', 'recommend', 'others', 'to', 'avoid', 'this', ',', 'movie', 'is', 'not', 'good', '.']


In [5]:
from sparknlp.pretrained import PretrainedPipeline 

In [6]:
pipeline = PretrainedPipeline('recognize_entities_dl', 'en')

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[OK!]


In [7]:
result = pipeline.annotate('Google has announced the release of a beta version of the popular TensorFlow machine learning library.')

In [8]:
print(result['ner'])
print(result['entities'])

['B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O']
['Google', 'TensorFlow']


In [10]:
!pip3 install spark-nlp-display

In [3]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pos = PerceptronModel.pretrained("pos_anc", 'en')\
        .setInputCols("document", "token")\
        .setOutputCol("pos")

chunker = Chunker()\
      .setInputCols(["document", "pos"])\
      .setOutputCol("chunk")\
      .setRegexParsers(["<NNP>+", "<DT>?<JJ>*<NN>"])


dep_parser = DependencyParserModel.pretrained('dependency_conllu')\
        .setInputCols(["document", "pos", "token"])\
        .setOutputCol("dependency")


typed_dep_parser = TypedDependencyParserModel.pretrained('dependency_typed_conllu')\
        .setInputCols(["token", "pos", "dependency"])\
        .setOutputCol("dependency_type")


nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          pos,
          dep_parser,
          typed_dep_parser
      ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)
lmodel = LightPipeline(pipelineModel)

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
dependency_typed_conllu download started this may take some time.
Approximate size to download 2.3 MB
[OK!]


In [16]:
text = ['Prime Minister Scott Morrison is standing by plans.']
res = lmodel.fullAnnotate(text)[0]
print(res)

{'dependency': [Annotation(dependency, 0, 4, Minister, {'head': '2', 'head.begin': '6', 'head.end': '13'}), Annotation(dependency, 6, 13, standing, {'head': '6', 'head.begin': '33', 'head.end': '40'}), Annotation(dependency, 15, 19, Minister, {'head': '2', 'head.begin': '6', 'head.end': '13'}), Annotation(dependency, 21, 28, Scott, {'head': '3', 'head.begin': '15', 'head.end': '19'}), Annotation(dependency, 30, 31, standing, {'head': '6', 'head.begin': '33', 'head.end': '40'}), Annotation(dependency, 33, 40, ROOT, {'head': '0', 'head.begin': '-1', 'head.end': '-1'}), Annotation(dependency, 42, 43, plans, {'head': '8', 'head.begin': '45', 'head.end': '49'}), Annotation(dependency, 45, 49, standing, {'head': '6', 'head.begin': '33', 'head.end': '40'}), Annotation(dependency, 50, 50, standing, {'head': '6', 'head.begin': '33', 'head.end': '40'})], 'dependency_type': [Annotation(labeled_dependency, 0, 4, flat, {}), Annotation(labeled_dependency, 6, 13, nsubj, {}), Annotation(labeled_depend

In [17]:
from sparknlp_display import DependencyParserVisualizer
dependency_vis = DependencyParserVisualizer()
dependency_vis.display(res, 'pos', 'dependency', 'dependency_type')
#if you get java memory heap error then need to increase the memory
#sudo vim $SPARK_HOME/conf/spark-defaults.conf
#uncomment the spark.driver.memory and change it according to your use. I changed it to below
#spark.driver.memory 15g

In [22]:
from sparknlp.pretrained import PretrainedPipeline

explain_document_pipeline = PretrainedPipeline("explain_document_ml")
annotations = explain_document_pipeline.annotate("We are very happy about SparkNLP")
print(annotations)


explain_document_ml download started this may take some time.
Approx size to download 9.1 MB
[OK!]


Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadPipeline.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 58.0 failed 1 times, most recent failure: Lost task 0.0 in stage 58.0 (TID 210) (192-168-1-109.tpgi.com.au executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2202)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2201)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2201)
	at or